CSE543 Algorithms for Nonlinear Optimization Course Project: 
RNN/LSTM Notebook

After adding APPL.csv to the notebook's directory, all cells can be rerun in current order to reproduce the results described in our paper for the RNN/LSTM model. 

In [ ]:
# import libaries 
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix
from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.utils import to_categorical

In [ ]:
# Functions to calculate RSI and MACD
def rsi(data, periods=14):
    delta = data['Close'].diff()
    gain, loss = delta.copy(), delta.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0

    avg_gain = gain.rolling(window=periods).mean()
    avg_loss = -loss.rolling(window=periods).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def macd(data, short=12, long=26, signal=9):
    exp1 = data['Close'].ewm(span=short, adjust=False).mean()
    exp2 = data['Close'].ewm(span=long, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

In [ ]:
# Load data and preprocess
#add APPL.csv to same directory as notebook
#data = pd.read_csv('/spy.csv')
data = pd.read_csv('/AAPL.csv')

#feature engineering
data['Average'] = (data['Open'] + data['Close'] + data['High'] + data['Low']) / 4
data['HL_PCT'] = (data['High'] - data['Low']) / data['Low']
data['PCT_change'] = (data['Close'] - data['Open']) / data['Open']
data['Volume_pct_change'] = data['Volume'].pct_change()
data['RSI'] = rsi(data)
data['MACD'], data['Signal'] = macd(data)

#target
forecast_out = 5
data['Target'] = data['Close'].shift(-forecast_out)

# Calculate the price direction (1 for gain, 0 for loss)
data['Price_Direction'] = (data['Target'] > data['Close']).astype(int)

# Remove rows with missing target values (last 5 days)
data.dropna(inplace=True)

# Split the data into features (X) and target (y) variables
X1 = data.drop(['Date', 'Target', 'Price_Direction'], axis=1).to_numpy()
y1 = data['Price_Direction'].to_numpy()

scaler = StandardScaler()
X1 = scaler.fit_transform(X1)

# Split data into train and test sets
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
#Setup LSTM timesteps
X = []
y = []

for i in range(5, len(X1)):
    X.append(X1[i-5:i,:])
    y.append(y1[i])
X, y = np.array(X), np.array(y)

In [ ]:
#ADAM with 0.001 initial learning rate
for i, (train_index, test_index) in enumerate(tscv.split(X)):
  X_train = X[train_index, :]
  X_test = X[test_index, :]
  y_train = y[train_index]
  y_test = y[test_index]

  # Scale the features

  model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  # Compile the model
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=50)

  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
  print(i)
  print(accuracy)
  yprob = model.predict(X_test)
  ypred = np.where(yprob > 0.5, 1, 0)
  print(np.count_nonzero(ypred < 0.5))
  print(np.count_nonzero(ypred > 0.5))
  accuracy = accuracy_score(y_test, ypred)
  roc_auc = roc_auc_score(y_test, yprob)
  f1 = f1_score(y_test, ypred)
  precision = precision_score(y_test, ypred)
  recall = recall_score(y_test, ypred)
  conf_matrix = confusion_matrix(y_test, ypred)
  print(accuracy)
  print(roc_auc)
  print(f1)
  print(precision)
  print(recall)
  print(conf_matrix)

In [ ]:
#ADAM with 0.0001 initial learning rate
for i, (train_index, test_index) in enumerate(tscv.split(X)):
  X_train = X[train_index, :]
  X_test = X[test_index, :]
  y_train = y[train_index]
  y_test = y[test_index]

  # Scale the features

  model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  # Compile the model
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=50)

  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
  print(i)
  print(accuracy)
  yprob = model.predict(X_test)
  ypred = np.where(yprob > 0.5, 1, 0)
  print(np.count_nonzero(ypred < 0.5))
  print(np.count_nonzero(ypred > 0.5))
  accuracy = accuracy_score(y_test, ypred)
  roc_auc = roc_auc_score(y_test, yprob)
  f1 = f1_score(y_test, ypred)
  precision = precision_score(y_test, ypred)
  recall = recall_score(y_test, ypred)
  conf_matrix = confusion_matrix(y_test, ypred)
  print(accuracy)
  print(roc_auc)
  print(f1)
  print(precision)
  print(recall)
  print(conf_matrix)

Epoch 1/50
52/52 [==============================] - 18s 22ms/step - loss: 0.6931 - accuracy: 0.5139
Epoch 2/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6931 - accuracy: 0.5055
Epoch 3/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6930 - accuracy: 0.5139
Epoch 4/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6930 - accuracy: 0.5055
Epoch 5/50
52/52 [==============================] - 2s 46ms/step - loss: 0.6928 - accuracy: 0.5091
Epoch 6/50
52/52 [==============================] - 2s 38ms/step - loss: 0.6928 - accuracy: 0.5261
Epoch 7/50
52/52 [==============================] - 2s 35ms/step - loss: 0.6928 - accuracy: 0.5200
Epoch 8/50
52/52 [==============================] - 2s 31ms/step - loss: 0.6923 - accuracy: 0.5345
Epoch 9/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6920 - accuracy: 0.5315
Epoch 10/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6917 - accuracy: 0.5333
Epoch 11

In [ ]:
#SGD with 0.001 initial learning rate
for i, (train_index, test_index) in enumerate(tscv.split(X)):
  X_train = X[train_index, :]
  X_test = X[test_index, :]
  y_train = y[train_index]
  y_test = y[test_index]

  # Scale the features

  model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  # Compile the model
  model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=50)

  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
  print(i)
  print(accuracy)
  yprob = model.predict(X_test)
  ypred = np.where(yprob > 0.5, 1, 0)
  print(np.count_nonzero(ypred < 0.5))
  print(np.count_nonzero(ypred > 0.5))
  accuracy = accuracy_score(y_test, ypred)
  roc_auc = roc_auc_score(y_test, yprob)
  f1 = f1_score(y_test, ypred)
  precision = precision_score(y_test, ypred)
  recall = recall_score(y_test, ypred)
  conf_matrix = confusion_matrix(y_test, ypred)
  print(accuracy)
  print(roc_auc)
  print(f1)
  print(precision)
  print(recall)
  print(conf_matrix)

Epoch 1/50
52/52 [==============================] - 12s 23ms/step - loss: 0.6933 - accuracy: 0.4818
Epoch 2/50
52/52 [==============================] - 1s 22ms/step - loss: 0.6933 - accuracy: 0.4848
Epoch 3/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6933 - accuracy: 0.5030
Epoch 4/50
52/52 [==============================] - 2s 30ms/step - loss: 0.6933 - accuracy: 0.4836
Epoch 5/50
52/52 [==============================] - 2s 34ms/step - loss: 0.6931 - accuracy: 0.5048
Epoch 6/50
52/52 [==============================] - 1s 25ms/step - loss: 0.6932 - accuracy: 0.4988
Epoch 7/50
52/52 [==============================] - 1s 22ms/step - loss: 0.6933 - accuracy: 0.5006
Epoch 8/50
52/52 [==============================] - 1s 24ms/step - loss: 0.6932 - accuracy: 0.4915
Epoch 9/50
52/52 [==============================] - 1s 22ms/step - loss: 0.6932 - accuracy: 0.5036
Epoch 10/50
52/52 [==============================] - 1s 24ms/step - loss: 0.6934 - accuracy: 0.5030
Epoch 11

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


104/104 [==============================] - 14s 31ms/step - loss: 0.6931 - accuracy: 0.4998
Epoch 2/50
104/104 [==============================] - 3s 25ms/step - loss: 0.6932 - accuracy: 0.5065
Epoch 3/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6930 - accuracy: 0.5056
Epoch 4/50
104/104 [==============================] - 2s 22ms/step - loss: 0.6932 - accuracy: 0.5053
Epoch 5/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6932 - accuracy: 0.5068
Epoch 6/50
104/104 [==============================] - 3s 29ms/step - loss: 0.6931 - accuracy: 0.5071
Epoch 7/50
104/104 [==============================] - 3s 26ms/step - loss: 0.6931 - accuracy: 0.5017
Epoch 8/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6932 - accuracy: 0.5065
Epoch 9/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6933 - accuracy: 0.5056
Epoch 10/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6932 - accuracy: 0.5065

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


155/155 [==============================] - 15s 24ms/step - loss: 0.6933 - accuracy: 0.4933
Epoch 2/50
155/155 [==============================] - 4s 23ms/step - loss: 0.6931 - accuracy: 0.5042
Epoch 3/50
155/155 [==============================] - 4s 23ms/step - loss: 0.6932 - accuracy: 0.5020
Epoch 4/50
155/155 [==============================] - 5s 29ms/step - loss: 0.6932 - accuracy: 0.5047
Epoch 5/50
155/155 [==============================] - 4s 23ms/step - loss: 0.6931 - accuracy: 0.5051
Epoch 6/50
155/155 [==============================] - 4s 23ms/step - loss: 0.6930 - accuracy: 0.5071
Epoch 7/50
155/155 [==============================] - 5s 29ms/step - loss: 0.6933 - accuracy: 0.5042
Epoch 8/50
155/155 [==============================] - 4s 24ms/step - loss: 0.6931 - accuracy: 0.5038
Epoch 9/50
155/155 [==============================] - 4s 23ms/step - loss: 0.6932 - accuracy: 0.4970
Epoch 10/50
155/155 [==============================] - 4s 25ms/step - loss: 0.6932 - accuracy: 0.5002

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


206/206 [==============================] - 18s 24ms/step - loss: 0.6932 - accuracy: 0.5084
Epoch 2/50
206/206 [==============================] - 5s 23ms/step - loss: 0.6931 - accuracy: 0.5087
Epoch 3/50
206/206 [==============================] - 6s 28ms/step - loss: 0.6932 - accuracy: 0.5084
Epoch 4/50
206/206 [==============================] - 5s 23ms/step - loss: 0.6931 - accuracy: 0.5080
Epoch 5/50
206/206 [==============================] - 5s 25ms/step - loss: 0.6931 - accuracy: 0.5084
Epoch 6/50
206/206 [==============================] - 5s 26ms/step - loss: 0.6931 - accuracy: 0.5086
Epoch 7/50
206/206 [==============================] - 5s 22ms/step - loss: 0.6931 - accuracy: 0.5086
Epoch 8/50
206/206 [==============================] - 6s 28ms/step - loss: 0.6930 - accuracy: 0.5089
Epoch 9/50
206/206 [==============================] - 5s 23ms/step - loss: 0.6932 - accuracy: 0.5092
Epoch 10/50
206/206 [==============================] - 5s 23ms/step - loss: 0.6931 - accuracy: 0.5086

In [ ]:
#SGD with 0.01 initial learning rate
for i, (train_index, test_index) in enumerate(tscv.split(X)):
  X_train = X[train_index, :]
  X_test = X[test_index, :]
  y_train = y[train_index]
  y_test = y[test_index]

  # Scale the features

  model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  # Compile the model
  model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=50)

  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
  print(i)
  print(accuracy)
  yprob = model.predict(X_test)
  ypred = np.where(yprob > 0.5, 1, 0)
  print(np.count_nonzero(ypred < 0.5))
  print(np.count_nonzero(ypred > 0.5))
  accuracy = accuracy_score(y_test, ypred)
  roc_auc = roc_auc_score(y_test, yprob)
  f1 = f1_score(y_test, ypred)
  precision = precision_score(y_test, ypred)
  recall = recall_score(y_test, ypred)
  conf_matrix = confusion_matrix(y_test, ypred)
  print(accuracy)
  print(roc_auc)
  print(f1)
  print(precision)
  print(recall)
  print(conf_matrix)

Epoch 1/50
52/52 [==============================] - 14s 23ms/step - loss: 0.6933 - accuracy: 0.4788
Epoch 2/50
52/52 [==============================] - 1s 24ms/step - loss: 0.6931 - accuracy: 0.4994
Epoch 3/50
52/52 [==============================] - 1s 22ms/step - loss: 0.6932 - accuracy: 0.5012
Epoch 4/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6932 - accuracy: 0.5073
Epoch 5/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6932 - accuracy: 0.4927
Epoch 6/50
52/52 [==============================] - 1s 23ms/step - loss: 0.6932 - accuracy: 0.5006
Epoch 7/50
52/52 [==============================] - 1s 24ms/step - loss: 0.6932 - accuracy: 0.5048
Epoch 8/50
52/52 [==============================] - 1s 26ms/step - loss: 0.6932 - accuracy: 0.5091
Epoch 9/50
52/52 [==============================] - 2s 34ms/step - loss: 0.6932 - accuracy: 0.5079
Epoch 10/50
52/52 [==============================] - 2s 31ms/step - loss: 0.6931 - accuracy: 0.5030
Epoch 11

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


104/104 [==============================] - 13s 23ms/step - loss: 0.6932 - accuracy: 0.5035
Epoch 2/50
104/104 [==============================] - 4s 34ms/step - loss: 0.6932 - accuracy: 0.4989
Epoch 3/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6932 - accuracy: 0.5059
Epoch 4/50
104/104 [==============================] - 2s 24ms/step - loss: 0.6931 - accuracy: 0.5114
Epoch 5/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6932 - accuracy: 0.5038
Epoch 6/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6932 - accuracy: 0.4932
Epoch 7/50
104/104 [==============================] - 4s 34ms/step - loss: 0.6932 - accuracy: 0.4986
Epoch 8/50
104/104 [==============================] - 2s 24ms/step - loss: 0.6933 - accuracy: 0.4889
Epoch 9/50
104/104 [==============================] - 3s 24ms/step - loss: 0.6932 - accuracy: 0.5050
Epoch 10/50
104/104 [==============================] - 2s 23ms/step - loss: 0.6932 - accuracy: 0.5032

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


155/155 [==============================] - 18s 30ms/step - loss: 0.6932 - accuracy: 0.5010
Epoch 2/50
155/155 [==============================] - 4s 24ms/step - loss: 0.6932 - accuracy: 0.5036
Epoch 3/50
155/155 [==============================] - 4s 23ms/step - loss: 0.6932 - accuracy: 0.5079
Epoch 4/50
155/155 [==============================] - 4s 26ms/step - loss: 0.6932 - accuracy: 0.5032
Epoch 5/50
155/155 [==============================] - 4s 27ms/step - loss: 0.6932 - accuracy: 0.5049
Epoch 6/50
155/155 [==============================] - 4s 23ms/step - loss: 0.6932 - accuracy: 0.4998
Epoch 7/50
155/155 [==============================] - 4s 23ms/step - loss: 0.6932 - accuracy: 0.5047
Epoch 8/50
155/155 [==============================] - 5s 30ms/step - loss: 0.6932 - accuracy: 0.5024
Epoch 9/50
155/155 [==============================] - 4s 24ms/step - loss: 0.6932 - accuracy: 0.5038
Epoch 10/50
155/155 [==============================] - 4s 24ms/step - loss: 0.6932 - accuracy: 0.5059

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


206/206 [==============================] - 18s 23ms/step - loss: 0.6932 - accuracy: 0.5010
Epoch 2/50
206/206 [==============================] - 5s 25ms/step - loss: 0.6933 - accuracy: 0.5081
Epoch 3/50
206/206 [==============================] - 6s 27ms/step - loss: 0.6931 - accuracy: 0.5084
Epoch 4/50
206/206 [==============================] - 5s 24ms/step - loss: 0.6932 - accuracy: 0.5080
Epoch 5/50
206/206 [==============================] - 6s 29ms/step - loss: 0.6931 - accuracy: 0.5087
Epoch 6/50
206/206 [==============================] - 5s 23ms/step - loss: 0.6931 - accuracy: 0.5095
Epoch 7/50
206/206 [==============================] - 5s 25ms/step - loss: 0.6931 - accuracy: 0.5096
Epoch 8/50
206/206 [==============================] - 6s 27ms/step - loss: 0.6932 - accuracy: 0.5080
Epoch 9/50
206/206 [==============================] - 5s 24ms/step - loss: 0.6931 - accuracy: 0.5081
Epoch 10/50
206/206 [==============================] - 6s 28ms/step - loss: 0.6931 - accuracy: 0.5092